# From photo to piece


In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from snap_fit.config.snap_fit_config import get_snap_fit_paths
from snap_fit.image.process import find_black_regions, find_contours
from snap_fit.image.utils import (
    display_image,
    draw_black_regions,
    load_image,
    save_image,
    show_image_mpl,
    sort_rects,
)

In [ ]:
sf_paths = get_snap_fit_paths()
data_fol = sf_paths.data_fol
sample_fol = data_fol / "sample"

In [ ]:
# img_fn = "PXL_20241130_105107220.jpg"
# img_fn = "front_01.jpg"
img_fn = "back_proc_02.jpg"
# img_fn = "puzzle_pieces_01.jpeg"
img_fp = sample_fol / img_fn

In [ ]:
# blah use the sample load script

In [ ]:
original_image = load_image(img_fp)
show_image_mpl(original_image)

In [ ]:
black_regions = find_black_regions(original_image)
black_regions = sort_rects(black_regions)
for i, (x, y, w, h) in enumerate(black_regions[:10]):
    print(f"Black Region {i + 1}: x={x}, y={y}, width={w}, height={h}")

In [ ]:
annotated_image = draw_black_regions(original_image, black_regions[:10])
show_image_mpl(annotated_image)

In [ ]:
contours = find_contours(original_image)
for i, contour in enumerate(contours[:1]):
    print(f"Contour {i + 1}: {contour}")